In [21]:
import pandas as pd

# combine csvs into one.

df = pd.read_csv('Merged.csv', index_col=0)
df = df.drop(['OffenseCode', 'LocationStreetNumber', 'LocationDirectional', 'CreatedSource'], axis=1)
print(df.head())    # print first 5 rows to check stuff




                 IncidentFromTime IncidentToDate IncidentToTime  \
IncidentFromDate                                                  
1/1/10                    1:55:00         1/1/10        2:58:00   
1/1/10                    1:36:00            NaN            NaN   
1/1/10                    2:00:00         1/1/10        4:00:00   
1/2/10                    6:15:00            NaN            NaN   
1/3/10                    1:29:00            NaN            NaN   

                      Offense Description      CaseStatus  \
IncidentFromDate                                            
1/1/10             Miscellaneous Offenses  Closed/Cleared   
1/1/10             Miscellaneous Offenses  Closed/Cleared   
1/1/10            Larceny – From Building        Inactive   
1/2/10             Miscellaneous Offenses  Closed/Cleared   
1/3/10             Miscellaneous Offenses  Closed/Cleared   

                       CaseDisposition LocationCode PatrolZone  \
IncidentFromDate                    